<a href="https://colab.research.google.com/github/yiwenwangANU/pytorch_review/blob/main/Extra_Shakespeare_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch import nn
from torch.nn import functional as F
import os
from pathlib import Path
import requests

In [2]:
if torch.cuda.is_available():
  device = torch.device("cuda")
  print("GPU ready!")
else:
  device = "cpu"
  print("GPU not available!")

GPU ready!


##Data download and explore

In [3]:
data_dir = Path.home() / 'data'
os.makedirs(data_dir, exist_ok=True)

data_dir = os.path.join(data_dir, "input.txt")
if os.path.exists(data_dir):
  print("Data ready.")
else:
  print(f"Downloading training data to {data_dir}")
  url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
  torch.hub.download_url_to_file(url, data_dir)

100%|██████████| 1.06M/1.06M [00:00<00:00, 25.7MB/s]


In [4]:
with open(data_dir, 'r', encoding='utf-8') as f:
    text = f.read()
    print(f"length of dataset in characters: {len(text)}")
    print('First 300 chars:')
    print(text[:300])

length of dataset in characters: 1115394
First 300 chars:
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us


##Tokenization

In [5]:
vocab = sorted(list(set(text)))
vocab_size = len(vocab)
print(f'Number of unique chars: {vocab_size}')
print(vocab)

Number of unique chars: 65
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [6]:
charTotoken = {char:i for i,char in enumerate(vocab)}
tokenTochar = {i:char for i,char in enumerate(vocab)}

In [7]:
def encode(input):
  return [charTotoken[char] for char in input]

def decode(input):
  return ''.join([tokenTochar[token] for token in input])

In [8]:
print(encode('First Citizen:'))
print(decode(encode('First Citizen:')))

[18, 47, 56, 57, 58, 1, 15, 47, 58, 47, 64, 43, 52, 10]
First Citizen:


Check other tokenizer like gpt-tokenizer

In [9]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape)
print(data[:100])

torch.Size([1115394])
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


##Train test split

In [10]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [11]:
torch.manual_seed(42)
batch_size = 4
block_size = 8

def get_sample(batch_size, block_size):
  idxes = torch.randint(len(train_data)-block_size, (batch_size,))
  xb = torch.stack([train_data[idx:idx+block_size] for idx in idxes])
  yb = torch.stack([train_data[idx+1: idx+block_size+1] for idx in idxes])
  return xb, yb

xb, yb = get_sample(batch_size, block_size)
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[57,  1, 46, 47, 57,  1, 50, 53],
        [ 1, 58, 46, 43, 56, 43,  1, 41],
        [17, 26, 15, 17, 10,  0, 32, 53],
        [57, 58,  6,  1, 61, 47, 58, 46]])
targets:
torch.Size([4, 8])
tensor([[ 1, 46, 47, 57,  1, 50, 53, 60],
        [58, 46, 43, 56, 43,  1, 41, 39],
        [26, 15, 17, 10,  0, 32, 53,  1],
        [58,  6,  1, 61, 47, 58, 46,  0]])
----
when input is [57] the target: 1
when input is [57, 1] the target: 46
when input is [57, 1, 46] the target: 47
when input is [57, 1, 46, 47] the target: 57
when input is [57, 1, 46, 47, 57] the target: 1
when input is [57, 1, 46, 47, 57, 1] the target: 50
when input is [57, 1, 46, 47, 57, 1, 50] the target: 53
when input is [57, 1, 46, 47, 57, 1, 50, 53] the target: 60
when input is [1] the target: 58
when input is [1, 58] the target: 46
when input is [1, 58, 46] the target: 43
when input is [1, 58, 46, 43] the target: 56
when input is [1, 58, 46, 43, 56] the target: 43
when input is [1, 58, 46,

In [12]:
embedding = nn.Embedding(num_embeddings=10, embedding_dim=3)

# Input tensor (indices of words or items)
input_indices = torch.tensor([1, 5, 7, 7])

# Get embeddings
output = embedding(input_indices)
print(output)

tensor([[-1.6047,  1.7878, -0.4780],
        [ 1.7174,  0.3189, -0.4245],
        [ 0.3211, -0.8798, -0.6011],
        [ 0.3211, -0.8798, -0.6011]], grad_fn=<EmbeddingBackward0>)


##Bigram Model

In [13]:
torch.manual_seed(42)

# A Bigram Model predicts the next token based only on the previous token
# (i.e., it learns a probability distribution of token transitions).
class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    # each token directly reads off the logits for the next token from a lookup table
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
  def forward(self, x):
    return self.token_embedding_table(x)

model_bigram = BigramLanguageModel(vocab_size)

In [14]:
# use the last token in x to predict next token
# and loop to generate text at seq_len length
torch.manual_seed(42)
def generate(x, seq_len): # x-> shape(Batch size, 1)
  for _ in range(seq_len):
    logits = model_bigram(x) # shape(Batch size, Sequence length, Vocabulary size)
    logits_last = logits[:,-1,:] # shape(Batch size, Vocabulary size)
    logits_last_prob = F.softmax(logits_last, dim=-1) # shape(Batch size, Vocabulary size)
    token_next = torch.multinomial(logits_last_prob, num_samples=1) # shape(B, 1)
    x = torch.cat((x, token_next), dim=1) # (B, T+1)
  return x

In [15]:
# generate use untrained model to generate text
new_text = generate(torch.zeros(1, 1, dtype=torch.long), seq_len=100)
print(new_text)
print(decode(new_text.squeeze().numpy()))

tensor([[ 0, 25, 60, 15, 19, 27, 61, 34, 35, 42, 40, 49, 49, 17, 51, 50, 12, 35,
          7, 11,  8, 50, 49, 28, 38,  7, 23, 54, 25, 17, 59, 28, 30, 30, 49, 17,
          4, 28, 43,  8, 63, 49,  7, 46, 15, 33, 59, 60, 49, 16, 62, 42, 24,  9,
         62, 62, 17, 25, 59, 28, 40, 38, 28, 19, 37, 12, 46, 31,  7, 52, 15, 40,
         39, 55, 59, 36, 46, 15,  9, 56, 16, 21,  7, 54, 64, 29, 14, 50,  5, 26,
         62, 42, 34, 28, 40, 47, 56, 39, 55, 35, 42]])

MvCGOwVWdbkkEml?W-;.lkPZ-KpMEuPRRkE&Pe.yk-hCUuvkDxdL3xxEMuPbZPGY?hS-nCbaquXhC3rDI-pzQBl'NxdVPbiraqWd


In [16]:
optimizer = torch.optim.Adam(params=model_bigram.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

In [17]:
epoches = 10000
for epoch in range(epoches):
  model_bigram.train()
  xb, yb = get_sample(batch_size, block_size) # get random sample in each epoch
  logits = model_bigram(xb)
  B, T, C = logits.shape # change the shape to match the input shape of loss_fn
  logits = logits.view(B*T, C)
  targets = yb.view(B*T)
  loss = loss_fn(logits, targets)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if(epoch == 0):
    print(f'Logits shape: {logits.shape}')
  if(epoch % 1000 == 0):
    print(f'epoch: {epoch}, train loss: {loss.item():.2f}')


Logits shape: torch.Size([32, 65])
epoch: 0, train loss: 4.92
epoch: 1000, train loss: 4.04
epoch: 2000, train loss: 4.02
epoch: 3000, train loss: 3.09
epoch: 4000, train loss: 2.89
epoch: 5000, train loss: 2.84
epoch: 6000, train loss: 2.91
epoch: 7000, train loss: 2.61
epoch: 8000, train loss: 2.75
epoch: 9000, train loss: 2.20


In [18]:
# generate use trained model to generate text
print(decode(generate(torch.zeros(1, 1, dtype=torch.long), seq_len=500).squeeze().numpy()))


u set

'Tuce f y mWchhagen ath awo;mWCivede cr V:

Su olavethizeageefotraycegought han'd.
WAy?Q.
LI.
s d pl mure at;'d dsthintw.OY an ha?it
Ther w hadend al frouigunchethimumad il: s s
tP?. fof-Go
TV&res.

LAne t pe.
BYO;Buce.

SBRYOLOUKWJPS: sty hucoism e wiar plon w
H-JGSothath, as bllved, hathrtan mP

F'Tw, ouer Lot.OelaHEO w.
INGOPHcituENEngZHou mb'lat

Torc?ierit heshth hedamemo sat k?WA! taps marmon oofor amy!vyx-INws v3N mpt I fanes nvTr hon ch ttho o3qKqHERThotouptyor w, fail.
Jghe, nchy


##Single head self-attention

In [19]:
## lower triangle masking
## to remove the influence from tokens to previous ones
B, T, C = 4, 8, 32 # batch, context/sequence length, channel/embedding
x = torch.rand(B, T, C)

def apply_mask(wei):
  tri = torch.tril(torch.ones(T, T))
  wei = torch.masked_fill(wei, tri==0, float('-inf')) # for text generation
  wei = F.softmax(wei, dim=-1)
  return wei

wei = torch.zeros(T, T)
print(apply_mask(wei))

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


In [20]:
## key, query and affinity matrix
B, T, C = 4, 8, 32
x = torch.rand(B, T, C)

head_size = 16 # as embedding size
# match each token from embedding to head that answer who I am
key = nn.Linear(C, head_size, bias=False)
# match each token from embedding to head that ask what I want
query = nn.Linear(C, head_size, bias=False)

k = key(x) # (B, T, head_size)
q = query(x) # (B, T, head_size)

# affinity matrix indicates the relationship between tokens
affinity = q @ k.transpose(-2, -1) # (B, T, head) @ (B, head, T) -> (B, T, T)
print(apply_mask(affinity)) # after apply mask and softmax

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4937, 0.5063, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3967, 0.3963, 0.2071, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3245, 0.3005, 0.1547, 0.2204, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2849, 0.2798, 0.1079, 0.1990, 0.1283, 0.0000, 0.0000, 0.0000],
         [0.2095, 0.1815, 0.1066, 0.1718, 0.1195, 0.2110, 0.0000, 0.0000],
         [0.1611, 0.1636, 0.1027, 0.1520, 0.1101, 0.1518, 0.1586, 0.0000],
         [0.1405, 0.1399, 0.0696, 0.1324, 0.1055, 0.1166, 0.1403, 0.1552]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5196, 0.4804, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3663, 0.3821, 0.2516, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3204, 0.3101, 0.1756, 0.1939, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2909, 0.2359, 0.1385, 0.1462, 0.1884, 0.0000, 0.0000, 0.0000],
         [0.2147, 0.173

$$
\text{Attention}(Q, K, V) = \text{softmax} \left( \frac{Q K^T}{\sqrt{d_k}} \right) V
$$

In [21]:
k = torch.randn(B, T, head_size)
k.var()

tensor(0.9140)

In [22]:
q = torch.randn(B, T, head_size)
q.var()

tensor(0.9795)

In [23]:
affinity = q @ k.transpose(-2, -1) * head_size ** -0.5 # to keep the var
affinity.var()

tensor(0.8151)

In [24]:
# value and normalization
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
# match each token from embedding to head that related what can it provide
value = nn.Linear(C, head_size, bias=False)

k = key(x)
q = query(x)
v = value(x)

affinity = q @ k.transpose(-2, -1) / torch.sqrt(torch.tensor(head_size))
affinity = apply_mask(affinity) # (B, T, T)
out = affinity @ v # (B, T, head_size)
out[0]

tensor([[ 0.0799, -0.4369,  0.1157,  0.2735,  0.0314,  0.2840,  0.0345,  0.4093,
         -0.3660,  0.0103, -1.0115,  0.9016, -0.3275,  0.3434,  0.3896,  0.0851],
        [ 0.2351, -0.3106,  0.1230,  0.1948, -0.0116,  0.3260,  0.1782,  0.1483,
         -0.3691, -0.0219, -1.0119,  0.7793, -0.4661,  0.2363,  0.1934, -0.1231],
        [ 0.0040, -0.2202, -0.0062,  0.0647, -0.2189,  0.1161,  0.0124, -0.0499,
         -0.1839, -0.0039, -0.3031,  0.6311, -0.2707,  0.2361,  0.0855, -0.4001],
        [-0.0667, -0.1820, -0.0615,  0.2115, -0.2336, -0.0343, -0.1382, -0.1585,
         -0.1798,  0.1959, -0.0330,  0.2147, -0.1741,  0.0181, -0.0929, -0.0515],
        [ 0.1482,  0.0084, -0.0211, -0.1298, -0.1261,  0.0053,  0.1351, -0.3450,
         -0.2445,  0.0236, -0.2005,  0.2888, -0.5325, -0.0952, -0.0543, -0.3679],
        [ 0.1782,  0.1300, -0.1134, -0.1933, -0.2255,  0.0343,  0.1572, -0.3720,
         -0.1617,  0.0374, -0.2903,  0.1180, -0.4894, -0.1279, -0.0017, -0.2030],
        [ 0.0497,  0.1

In [46]:
batch_size = 32 # B
block_size = 8 # T
embedding_dims = 32 # C
vocab_size = 65
epoches = 5000
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [47]:
class Head(nn.Module):
  # single head self attention
  def __init__(self, block_size, embedding_dims, head_size):
    super().__init__()
    self.query = nn.Linear(embedding_dims, head_size)
    self.key = nn.Linear(embedding_dims, head_size)
    self.value = nn.Linear(embedding_dims, head_size)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

  def forward(self, x):
    B, T, C = x.shape

    q = self.query(x)
    k = self.key(x)
    v = self.value(x)
    affinity = q @ k.transpose(-2, -1) / torch.sqrt(torch.tensor(k.shape[-1]))

    affinity = torch.masked_fill(affinity, self.tril[:T, :T] == 0, float('-inf')) # for text generation
    affinity = F.softmax(affinity, dim=-1)

    out = affinity @ v # (B, T, T) @ (B, T, head_size) -> (B, T, head_size)
    return out

In [48]:
class SingleHeadModel(nn.Module):
  def __init__(self, vocab_size, embedding_dims, block_size):
    super().__init__()
    self.token_embedding = nn.Embedding(vocab_size, embedding_dims)
    self.positon_embedding = nn.Embedding(block_size, embedding_dims)
    self.self_attention_head = Head(block_size, embedding_dims, head_size=embedding_dims)
    self.decoder = nn.Linear(embedding_dims, vocab_size)

  def forward(self, x):
    B, T = x.shape
    x = self.token_embedding(x) + self.positon_embedding(torch.arange(T, device=device)) # (B, T, C) + (T, C) -> (B, T, C)
    x = self.self_attention_head(x) # (B, T, C)
    x = self.decoder(x) # (B, T, vocab_size)
    return x

model_single_head = SingleHeadModel(vocab_size=vocab_size, embedding_dims=embedding_dims, block_size=block_size).to(device)

In [73]:
# use x(shape(B, T)) to predict next token,
# after go through the model still use last token to predict the next token,
# and loop to generate text at new_seq_len length
def generate(x, new_seq_len): # x-> shape(B, T)
  model_single_head.eval()
  x = x.to(device)
  for _ in range(new_seq_len):
    with torch.inference_mode():
      context = x[:, -block_size:] # if x exceeds block_size, only feed the last block_size tokens to the model
      logits = model_single_head(context) # (B, T, vocab_size)
      logits_last = logits[:,-1,:] # (B, vocab_size)
      logits_last_prob = F.softmax(logits_last, dim=-1) # (B, vocab_size)
      token_next = torch.multinomial(logits_last_prob, num_samples=1) # (B, 1)
      x = torch.cat((x, token_next), dim=1) # (B, T+1)
  return x

In [50]:
optimizer = torch.optim.Adam(params=model_single_head.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

In [52]:
for epoch in range(epoches):
  model_single_head.train()
  xb, yb = get_sample(batch_size, block_size) # get random sample in each epoch
  xb, yb = xb.to(device), yb.to(device)
  logits = model_single_head(xb)
  B, T, C = logits.shape # change the shape to match the input shape of loss_fn
  logits = logits.view(B*T, C)
  targets = yb.view(B*T)
  loss = loss_fn(logits, targets)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if(epoch == 0):
    print(f'Logits shape: {logits.shape}')
  if(epoch % 1000 == 0):
    print(f'epoch: {epoch}, train loss: {loss.item():.2f}')

Logits shape: torch.Size([256, 65])
epoch: 0, train loss: 4.13
epoch: 1000, train loss: 2.39
epoch: 2000, train loss: 2.54
epoch: 3000, train loss: 2.27
epoch: 4000, train loss: 2.41
epoch: 5000, train loss: 2.37
epoch: 6000, train loss: 2.39
epoch: 7000, train loss: 2.38
epoch: 8000, train loss: 2.28
epoch: 9000, train loss: 2.39


In [74]:
# generate use trained model to generate text
print(decode(generate(torch.zeros(1, 1, dtype=torch.long), new_seq_len=500).squeeze().cpu().numpy()))


PERTANERI KIUSins ms innen,
Sis, ise ito mee the omaber nes doucrothu gem eperelis our'!

Wopere to thef ter fo heamal chag ds: nde 't thago!
Whichould by eave wis horu,
QUUKIUS:
O-tillepo mitothyu?

PELORLUKE IZARENRCHENTESS:
Whath mathy, dse an end al mane 'se;
USl:
An me;
G RCIFIO:
Oy, yofa hot haer spans ayt dom?

Fes I thereswanthe?-
Heckncere pu'd ar.

Hed foith mnpr tay aknifet
Bent pe:
He, RD IURESl: hath dthivowupnsem o's. ard prsithe lifucay hourig men.


foud thouth alwh ay wopron, th
